***CATEGORY THEORY WITH JULIA***
-----

# 3. Functors

In Category Theory, a functor $F$ is similar to a morphism, but while morphisms act on objects,
functors act on categories.

**Definition (Functor in Category Theory).** A functor is $F:\mathcal C \to \mathcal D$ such that
for every object $a \in Ob_{\mathcal C}$ there is $F(a) \in Ob_{\mathcal D}$. Also,

$$F(id_a) = id_{F(a)},$$

And for

$$f \in Mor_{\mathcal C}(a,b) \text{ and }g \in Mor_{\mathcal C}(c,a)$$

We have

$$F(f):Fa \to Fb \text{ and } F(g):Fc \to Fa \text{ such that }$$

$$F(f \circ g) = F f \circ Fg.$$


When a functor has the same category in the domain and codmain (i.e. $F:\mathcal C \to \mathcal C$) we call
it an **endofunctor**.

As we'll see, in programming, our functors will be endofunctors acting in the category of types.


## 3.1 Functors In Programming

So, what is this like in a programming language? 

A functor takes types and returns types, and it takes functions and returns functions, and lastly,
it must satisfy the composition property and the identity property.

In programming (and Julia more specifically), a functor is represented by a
parametric struct `F{T}`, a type constructor for such parametric struct, and an `fmap` function,
which is used to apply an arbitrary function `f` to an instance of `F{T}`.
Why this `fmap`? Because it takes the role of `F(f)`. We can build `F(f)` from `fmap`.

Let's do an example below.

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `~/MEGA/EMAP/PhDThesis/notes/FunctionalProgrammingCategoryTheory/julia`


In [2]:
struct F{T}
    x::T
    y::T
end

# This is the type constructor
F(T::Type) = F{T}

fmap(f::Function, a::F{T}) where T = F(f(a.x), f(a.y))

# Note that from fmap we can do
F(f::Function) = a -> fmap(f,a);

# CAUTION! Overloading `F` in such a way may cause problems when, for example,
# your struct has only a single field. In this case, if you wanted to create
# a struct where the element was a function, this implementation would enter in conflict

So why is this implementation a functor? Because, `F` defines how to take objects (types) `T`
and maps them to type `F{T}`. Hence, we have the part acting on the objects. But functors
also act on functions. This behaviour is coded in the `fmap`. Note,
`fmap(f,F(a)) ≡ F(f) (F(a))`.

But what about the composition and identity preservation? That depends on the implementation
of the `fmap`. Fortunantly, it can be shown that our implementation does satisfies these properties.
We'll not prove it, instead, we'll just show an example.

In [3]:
a = F(1,2)

id(x) = x
f(x)  = √2
g(x)  = x^2


@show (F(f) ∘ F(g))(a) == F(f ∘ g)(a)
@show F(id)(a);

(F(f) ∘ F(g))(a) == (F(f ∘ g))(a) = true
(F(id))(a) = F{Int64}(1, 2)


*There is a catch in the example above*.

In the example above, when we created our struct, Julia implicitly created for us a "value" constructor.
"How and where?" you might ask. Note, when you write `F(1,2)`, Julia is actually creating a value of
type `F{Int}`. This value constructor is **not** part of the definition of a functor.
A functor does not say how to select values from types.

We actually already explained this in section *1 - Types*. This value constructor is a morphism
`F(x::T, y::T) where T` to a value of type `F{T}`. But there is a difference here! The type `T`
is not an actual type, but a parameter. And this parameter is actually inferred from the types
of `x` and `y`.

<!-- So if this is not part of the functor, than what is it? In Category Theory, this value selector
can be defined as a natural transformation. We'll go deeper into natural transformations latter, but let's
give a brief introduction here.

Natural transformations are like morphisms, but between functors. Let $F:\mathcal C \to \mathcal C$
and $G:\mathcal C \to \mathcal C$ be two endofunctors. A natural transformation between these functors is
$\eta:F\to G$ such that for every $A \in \mathcal C$ we have $\eta_A:FA \to GA$. Hence, a natural
transformation defines a function that selects an element of $GA$ for every element of $FA$ that we pass,
for any type $A$.

In the example above, `F(1,2)` is equivalent to $eta_{Tuple{Int,Int}}(Tuple(1,2)) = F(1,2)$. -->

## 3.2 The `Maybe` Functor

Another common example of functor is the `Maybe` functor. From a type `T`, the functor
`Maybe` returns a type `Union{T, Nothing}`.

What about the function mapping? In that case, `Maybe f` is a function that returns
`nothing` if it gets `nothing` as input. Otherwise, it returns the same result as `f`.

Below is an implementation.

In [4]:
Maybe{T} = Union{T, Nothing}
Maybe(T::Type) = Union{T, Nothing}

fmap(f::Function, x::Maybe) = x === nothing ? nothing : f(x)

f(x) = x^2
@show fmap(f,nothing);
@show fmap(f,2);

fmap(f, nothing) = nothing
fmap(f, 2) = 4


Note that in this case we **didn't even implement the struct**. Why?
Because Julia already has the `Union` type, hence, we just used the so called *type alias*,
meaning, we just created another alias (name) for the `Union{T, Nothing}` type.
This type already defines a way of says that a value is either of type `T`or type `Nothing`.

Our `fmap` was also a bit different. Yet, it's simple to understand what is going on.
The `fmap` takes a function and applies it to `x` of type `Maybe`. Now, if `x` is
`nothing`, then the function returns nothing. Otherwise, it just applies `f` to `x`.  

This implementation is very different from what, for example, is described in Haskell.
In Haskell, this would be implemented as something like:

In [78]:
abstract type Maybe2{T} end

struct Just{T} <: Maybe2{T}
    _1::T
end

struct nil{T} <: Maybe2{T} end

Maybe2(T::Type) = Maybe2{T}
fmap(f::Function, x::Maybe2{T}) where T = x isa nil ? nil{T}() : Just(f(x._1))

@show Just(1) isa Maybe2, nil{Any}() isa Maybe2
fmap(x-> x^2, Just(2)), fmap(x->x^2, nil{Any}())

(Just(1) isa Maybe2, nil{Any}() isa Maybe2) = (true, true)


(Just{Int64}(4), nil{Any}())

## 3.3 The `Vector` Functor

Note that `Vector{T}` works like a functor. By passing a type say `Int`, we get a type Vector{Int}.
The fmap for a `Vector{T}` is just passing a function to each of it's elements.
Again, one can prove that such implementation satisfies the composition and identity properties.

In the implementation below, we use the name `Vec` to avoid clashing with the name `Vector`
which is already from Julia's Base.

In [79]:
fmap(f,v::Vector{T}) where T = [f(i) for i in v]

Vector(x::T...) where T = [x...]

Vec(f) = v -> [f(i) for i in v]

Vec(x->x^2)([2,2])

2-element Vector{Int64}:
 4
 4

## 3.4 The `Const{T}` Functor

Let's start with an example. The `Const_Int` functor is a functor
that takes any type `T` and returns the type `Int`.
This functor takes every morphism (function)
to the identity morphism of `Int`. Note, if `f(x::A)::B`, then `Const{Int}(A)=Int`
and `Const{Int}(B) = Int`. Hence, `Const{Int}(f) = x::Int->x`.

In [80]:
Const_Int(T::Type) = Int

Const_Int(f::Function) = x::Int -> x

Const_Int (generic function with 2 methods)

Again, we've constructed our functor without the `fmap`. For this to be possible, we would need to
create a struct.

Now, note that `Const_Int` can be generalized if we use type parameters. Meaning, we can
create a whole class of `Const` functors.

In [81]:
struct Const{T} end
Const{T}(A::Type) where T = T

Const{T}(f::Function) where T = x::T -> x
fmap(f::Function, x::Const{T}) where T = x

fmap(x->x^2, Const{Int}())

Const{Int}(String)

Int64

Note that `Const{Int}` is equivalant to the functor `Const_Int`, in the sense that it takes
every type `A` to type `Int`, and takes every function `f(x::A)::B` to `id(x::Int) = x`.

There is a strange abstraction going on here. The only "value" of type `Const{T}`
is `Const{T}()`.

It might be useful to actually store a value inside of this `Const{T}` struct.
Yet, once we do this, we are actually defining more than the functor, since now,
we have an element for each storage value.

In [82]:
struct ConstStore{T}
    value::T
end

ConstStore{T}(A::Type) where T = T
ConstStore{T}(f::Function) where T = x::T -> x

fmap(f::Function, x::ConstStore{T}) where T = x

fmap(x->10, ConstStore(1))

ConstStore{Int64}(1)

## 3.5 List Functor

The `Vector{T}` functor is already a list functor. But let's implement our own.

In [83]:
abstract type List{T} end
struct Nil{T} <: List{T} end
struct Cons{T} <: List{T}
  val::T
  next::List{T} # n.b. abstract type!
end
Nil(T) = Nil{T}()
Nil() = Nil(Any)

## Example of list
Cons(1, Cons(1,Nil{Int}()))

## Another example of list
Cons(x::T) where T = Cons(x, Nil(T))
l = Cons(1,Cons(2));


fmap(f::Function, x::Nil) = x
fmap(f::Function, x::Cons{T}) where T = Cons(f(x.val),fmap(f, x.next))

fmap(x->10,l)

Cons{Int64}(10, Cons{Int64}(10, Nil{Int64}()))

## 3.6 Functor Composition

In the same way that we compose morphisms (functions) we can think about composing functors
when the domain and codomain match. Yet, functors act not only on objects, but also morphisms. Hence,
how should the composition work. Here it is:

Let $F:\mathcal C \to \mathcal D$ and $G:\mathcal D \to \mathcal E$ be two functors. We define
* For any $A \in \mathcal C$, then $(G \circ F) A = G(F(A))$;
* For any $f \in \mathcal C(A,B)$, then $(G \circ F) f = G(F(f))$.

For such composition, an identity functor $I:\mathcal D \to \mathcal D$ is a functor
that for $A \in \mathcal D$ we have $I(A) = A$, and $If(A) = f(A)$.

Since in programming our functors are endofunctors ($F:\mathcal C \to \mathcal C$),
then they are composable.

Let's do an example by implementing a function `maybetail`
that takes a `List` and returns just the "tail".

In [84]:
# Since `Nil` and `Cons` are the only subtypes of `List`, then `maybetail` can
# be thought of as a function from List.
maybetail(x::Nil) =  nil{List}()
maybetail(x::Cons) = Just(x.next)


l = Cons(1,Cons(2,Cons(3)));

maybetail(l), maybetail(Nil())

(Just{Cons{Int64}}(Cons{Int64}(2, Cons{Int64}(3, Nil{Int64}()))), nil{List}())

Note that this implementation returns a `Maybe2{List}`, since it takes `List` and
returns either `nil{List}` or `Just{List}`, which as the subtypes of `Maybe2{List}`.

The question now is, how do we use `fmap` in such case?
For example:

In [85]:
mis = Just(l)

Just{Cons{Int64}}(Cons{Int64}(1, Cons{Int64}(2, Cons{Int64}(3, Nil{Int64}()))))

How do we take the squares of each value in this list? I.e. how do we compose these functors?
The answer is in the example below:

In [86]:
sq(x) = x^2

fmap(x->fmap(sq,x),mis)

Just{Cons{Int64}}(Cons{Int64}(1, Cons{Int64}(4, Cons{Int64}(9, Nil{Int64}()))))

## 3.7 Bifunctor 

A functor takes one type as argument. The bifunctor takes two.

In [26]:
struct Bifunctor{R,L}
    x::R
    y::L
end

Bifunctor(::Type{R},::Type{L}) where R where L = Bifunctor{R,L}

map_Bifunctor(f,g) = b -> Bifunctor(f(b.x), g(b.y))

b = Bifunctor(2,"y")

map_Bifunctor(x->√x,x->uppercase(x))(b)

Bifunctor{Float64, String}(1.4142135623730951, "Y")

## 3.8 Generating Functors

As we've seen, given a parametric struct `S{T}`, there is a corresponding functor is just
the type constructor and the lifting functor, that takes a function f a applies it to each field
in the struct.

Hence, one can "automatically" generate the respective functor for each struct.
In Haskell, this is done by adding  `{-# LANGUAGE DeriveFunctor #-}` to the code, and hence
by running:
```
data Maybe a = Nothing | Just a deriving Functor
```

The corresponding fmap will be generated.

In Julia, we can use the Functors.jl package.

In [ ]:
using Functors
struct MyType{A,B,C}
    x::A
    y::B
    w::B
    z::C
end

t = MyType(1,1.0,2.0,"t")
fmap(string, t)

Note, this is not the correct behavior! The fmap applied a string to the whole
thing, and not to the elements. The reason is that we haven't overloaded
the fmap function for MyType. This will be done using the @functor macro from Functors.jl.

In [ ]:
@functor MyType
t = MyType(1,1.0,2.0,"t")
fmap(string, t)